This code is used for the capstone project workflow:
1. Download CNVs of interest from dbVar
2. **Extract UIDs (unique identifiers)**
3. Use eSummary to extract CNV summaries
4. **Extract genomic coordinates from summaries**
5. Perform manual CNV search in Mastermind
6. Perform custom data analysis and create visualizations

This file includes the code for the steps in bold. The other steps are (currently) manual

#Step 1: Doanload CNVs of Interest from dbVar

Begin by selecting a subset of the CNVs in dbVar to analyze by using the advanced search feature in dbVar.

**Example advanced search (for pathogenic gain CNVs):** 

(((((("copy number gain"[Variant Type] AND "pathogenic"[Clinical Interpretation]) NOT "copy number loss"[Variant Type]) NOT "uncertain significance"[Clinical Interpretation]) NOT "benign"[Clinical Interpretation]) NOT "likely benign"[Clinical Interpretation]) NOT "likely pathogenic"[Clinical Interpretation]) NOT "conflicting interpretations of pathogenicity"[Clinical Interpretation] AND ("VARIANT"[OBJ_TYPE] AND "copy number variation"[Variant Type] AND "Pathogenic"[clinical_interpretation] AND 1[has_clinical])

# Step 2: Extract UIDs

Run the code below (replace file_name with your file path/name)

In [ ]:
def UID_extractor_pathogenic(path):
# Input: A file path for a file downloaded from NCBI's dbVar database
# Output: A list of 200 randomly selected UIDs for all the CNVs in the original file

  # Read in the file
  data_file = open(path,'r')
  data = data_file.readlines()
  data_file.close()

  # Extract UIDs
  UIDs = []   # Start by making a new list for all the UIDs
  for i in range(len(data)):
    if data[i][0:33] == 'Clinical significance: Pathogenic':
      UIDs.append(data[i+2].strip())
  # Get rid of 'ID: ' in each element of the list
  UIDs_clean = []
  for s in UIDs:
    UIDs_clean.append(s[4:])
  print('Length of full UIDs list:',len(UIDs_clean))  # Make sure it worked as intended
  print('First 10 UIDs:',UIDs_clean[0:10]) # Make sure it worked as intended

  # Select 200 random UIDs to pass into the next part of the workflow
  UIDs200 = []
  count = 0
  while count < 200:
    random_UID = UIDs_clean[randint(0,len(UIDs_clean)-1)]
    if random_UID not in UIDs200:
      UIDs200.append(random_UID)
      count += 1
  print("Length of UIDs200 (should be 200): ",len(UIDs200))   

  # Return a list of UIDs
  final_list = ",".join(UIDs200)
  return final_list

In [ ]:
# import random number generator
from random import seed
from random import randint
# seed random number generator
seed(1)

In [ ]:
# Run UDF on pathogenic losses file
file_name = '/content/drive/MyDrive/Capstone Scripts/pathogenic_losses.txt' # replace with your file path/name
UID_extractor_pathogenic(file_name)

Length of UIDs list: 6763
First 10 UIDs: ['48438691', '53677449', '48484688', '48460685', '48441091', '52235360', '50454324', '48460610', '48485842', '48467902']
Length of UIDs200:  200


'48436665,53677353,50454366,48477854,50290265,48478988,47178757,50273201,48479485,48474924,48480686,48473240,48483658,48472497,50373014,49622806,48477774,48474510,48447923,48460551,48460567,48456403,54356099,48466869,48478357,48444934,48478293,48468214,48457484,48452042,48470456,48485437,48457247,50286953,48474229,49621112,48472413,50271920,49622068,50372182,48456324,49623360,48441038,49343744,53679220,48468545,48460613,50372047,49616937,48479757,48472354,48475665,48439176,53680318,17336739,48439998,48454915,48474140,48487320,54348714,48478407,49621028,46810648,48474478,48483325,48450022,47178742,48454836,54356371,48484928,48461008,48476043,49621544,48454822,48467555,48482814,50272989,50272026,48463131,48486439,48454858,48448424,48486675,48450149,48487654,53150651,48452138,48454554,48460957,53636913,48476519,48483305,48478491,53680510,53680287,48484736,48459931,52235366,48453737,48465085,48481979,48470336,48468093,54355583,48442392,48461160,48453533,48481557,48479282,50290726,50454357,

# Step 3: Use eSummary to extract CNV summaries

Paste the list of UIDs from the end of the previous step at the end of the following URL to perform the eSummary query:

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=dbvar&id=

The output should be a text file containing CNV summaries


# Step 4: Extract genomic coordinates from summaries

The text file we are looking at represents the following heirarchy:

\<eSummaryResult>
- \<DocumentSummarySet status="OK">
  - \<DbBuild>Build230209-1305.1\</DbBuild>
  - **\<DocumentSummary uid="48459297">** (first line for each CNV)
    - \<OBJ_TYPE>VARIANT\</OBJ_TYPE>
    - \<ST> (Study ID, e.g. nstd102)
    - \<SV> (variant ID, e.g. nsv3895942)
    - \<dbVarPlacementList> (list of coordinate info)
      - \<dbVarPlacement>
        - has \<Chr>,\<Chr_start>,\<Chr_end>,\<Assembly>
    - \<dbVarGeneList> (list of gene IDs and names)
      - \<dbVarGene> (contains \<id> and \<name>)
    - \<dbVarMethodList>
    - \<dbVarClinicalSignificanceList>
    - \<dbVarVariantTypeList>
    - \<Variant_call_count>
    - \<SortOrder>
  - **\</DocumentSummary>** (last line for each CNV)
  - *more CNV records with the same structure shown above...*
- \</DocumentSummarySet>

\</eSummaryResult>



Note: The following UDF uses the *dup_or_del* parameter which is customized for the way it was used in my capstone project. This parameter may need to be replaced/modified depending on its intended purpose.

In [ ]:
import re

def coordinate_extractor_pathogenic(path,dup_or_del):
# Input: A path for a file containing the results of an eSummary call using E-Utilities.
#        dup_or_del should be 0 if CNVs are duplications and 1 if CNVs are deletions
# Output: Prints list of UIDs, genomic coordinates (GRCh37), and Mastermind queries for the CNVs.
#         Returns the number of CNVs with GRCh37 coordinates from the input file.

  # Read the data from the file into a list
  data_file = open(path,'r') 
  data = data_file.read().split()
  data_file.close()
  stripped_data = [line.strip('\\') for line in data] 
  # print(stripped_data)

  # Extract genomic coordinates
  docSum_list = []         # list which will contain all Document Summaries as individual strings
  continue_adding = False  # boolean that signifies whether we should continue adding to the docsum string
  currentDocSum = ''       # string representing the current DocumentSummary (to be added to the list)

  for line in stripped_data:
    if not continue_adding and 'uid=' in line: # if the first line of a DocumentSummary is reached
      continue_adding = True
      currentDocSum = currentDocSum + line
    elif continue_adding:                      # if we are adding to the currentDocSum string
      currentDocSum = currentDocSum + line
      if '</DocumentSummary' in line:          # if the last line of the DocumentSummary is reached
        docSum_list.append(currentDocSum)
        continue_adding = False
        currentDocSum = ''

  final_list = []     # this list will contain lists of variant IDs and GRCh37 coordinates for each CNV record
  missing_GRCh37 = 0  # this is a counter to keep track of how many CNVs don't have GRCh37 coordinates (e.g. they may have GRCh37.p13 coordinates instead)

  regexpID = re.compile('uid=\"(?P<id>\d*)\"')
  regexpCoordinates = re.compile('<dbVarPlacement><Chr>(?P<chr>[\d|X|Y]+)</Chr><Chr_start>(?P<start>\d+)</Chr_start><Chr_end>(?P<end>\d+)</Chr_end><Assembly>GRCh37</Assembly>')
  
  for docsum in docSum_list:
    regexpID_match = regexpID.search(docsum)
    regexpCoordinates_match = regexpCoordinates.search(docsum)
    if regexpID_match and regexpCoordinates_match:          # if there is a match
      id = regexpID_match.group('id')
      chr = regexpCoordinates_match.group('chr')
      start = regexpCoordinates_match.group('start')
      end = regexpCoordinates_match.group('end')
      final_list.append([id,chr,start,end])
    else:
      print(regexpID_match.group('id') + ' does not have GRCh37 coordinates')
      missing_GRCh37 += 1

  print('\nCNVs without GRCh37 coordinates: ',missing_GRCh37,'\n')
  print('\n\n\nUIDs:')
  uid_list = [print(record[0]) for record in final_list]
  print('\n\n\nChromosomes:')
  chr_list = [print(record[1]) for record in final_list]
  print('\n\n\nStart Positions:')
  chr_list = [print(record[2]) for record in final_list]
  print('\n\n\nEnd Positions:')
  chr_list = [print(record[3]) for record in final_list]
  print('\n\n\nMastermind Queries:')
  if dup_or_del == 0:
    query_list = [print('duplication (chr' + record[1] + ':' + record[2] + '-' + record[3] + ' (GRCh37)),') for record in final_list]
  elif dup_or_del == 1:
    query_list = [print('deletion (chr' + record[1] + ':' + record[2] + '-' + record[3] + ' (GRCh37)),') for record in final_list]
  else:
    print("Warning: Duplication or Deletion parameter has not been properly specified!")
  
  return len(final_list)

In [ ]:
# Run the UDF on your file
file_name2 = '/content/drive/MyDrive/Capstone Scripts/eSummary200PathogenicLosses.txt' # replace with your file path/name
coordinate_extractor_pathogenic(file_name2,1) # May need to adjust 2nd parameter depending on the intended use

# Step 5: Perform manual CNV search in Mastermind

Paste the Mastermind queries from the previous step into Mastermind in order to obtain the metrics of interest. For my project, I recorded the GPO values (defined in the Methods section), type of overlap, the number of exact matches (if any), and the match terms (for CNVs with 10+ matches or exact matches)

# Step 6: Perform custom data analysis and create visualizations

See Figures_Script.ipynb file